!curl https://course.fast.ai/setup/colab | bash

In [1]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [2]:
x_train,y_train,x_val,y_val = get_data()

In [3]:
train_mean, train_std = x_train.mean(),x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [4]:
x_train.shape

torch.Size([50000, 784])

In [ ]:
x_train = normalize(x_train,train_mean,train_std)
y_train = normalize(y_train,train_mean,train_std)